In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime
import time
import pandas as pd

options = Options()
# options.add_argument('--headless')  # 디버깅 시 끄기
options.add_argument('--start-maximized')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

all_data = []

for page in range(34):  
    url = f'https://www.usda.gov/search?query=robusta&commit=Search&page={page}'
    print(f"[페이지 {page}] 크롤링 중: {url}")
    driver.get(url)
    time.sleep(8)  

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.select("div.views-row")

    if not articles:
        print(f"⚠ 페이지 {page}에서 기사 항목을 찾을 수 없습니다. 건너뜁니다.")
        continue

    for article in articles:
        title_tag = article.select_one('div.node-teaser__content h2 a')
        date_tag = article.select_one('span.date.text-bold')

        if title_tag and date_tag:
            title = title_tag.get_text(strip=True)
            link = title_tag['href']
            full_url = f"https://www.usda.gov{link}" if link.startswith("/") else link

            raw_date = date_tag.get_text(strip=True)

            # 날짜 변환
            try:
                date = datetime.strptime(raw_date, "%B %d, %Y").strftime("%Y-%m-%d")
            except ValueError:
                try:
                    date = datetime.strptime(raw_date, "%b %d, %Y").strftime("%Y-%m-%d")
                except:
                    date = raw_date

            all_data.append({
                "date": date,
                "title": title,
                "url": full_url
            })

driver.quit()

df = pd.DataFrame(all_data)

# 날짜 기준 내림차순 정렬 (최신순)
df = df.sort_values(by='date', ascending=False)
df.to_csv("usda_robusta_articles.csv", index=False, encoding='utf-8-sig')

print(f"수집 완료: 총 {len(df)}개 기사 → 'usda_robusta_articles.csv' 저장됨")

[페이지 0] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=0
[페이지 1] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=1
[페이지 2] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=2
[페이지 3] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=3
[페이지 4] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=4
[페이지 5] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=5
[페이지 6] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=6
[페이지 7] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=7
[페이지 8] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=8
[페이지 9] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=9
[페이지 10] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=10
[페이지 11] 크롤링 중: https://www.usda.gov/search?query=robusta&commit=Search&page=11
[페이지 12] 크롤링 중: https://www.usda.gov/search?query=robusta&co